In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.tree import plot_tree

In [5]:
def generate_sample(N = 30):
    # generate a sample of size N = 30, with two classes and 5 features 
    # each having a standard Gaussian distribution with pairwise correlation 0.95
    num_features = 5
    corr_matrix = np.ones((num_features, num_features)) * 0.95 + np.identity(num_features) * 0.05
    X = np.random.multivariate_normal(mean=np.zeros(num_features), cov=corr_matrix, size=N)

    # The response Y was generated according to Pr(Y = 1|x1 ≤ 0.5) = 0.2, Pr(Y = 1|x1 > 0.5) = 0.8
    # what is the lowest possible error rate?
    Y = np.zeros(N)
    for i in range(N):
        if X[i,0] <= 0.5: # 80% to be 0
            Y[i] = np.random.binomial(1, 0.2)
        else: # 80% to be 1
            Y[i] = np.random.binomial(1, 0.8) 

    return X, Y

X_train, y_train = generate_sample(N = 40) # we need some variabiliy in the data for bootstrap to work
X_test, y_test = generate_sample(N = 2000)

treeclf = DecisionTreeClassifier(random_state=1)
treeclf.fit(X_train, y_train)
y_train_pred = treeclf.predict(X_train)
y_pred = treeclf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(accuracy_score(y_train, y_train_pred)))
print('The accuracy on test data is {:.2f}'.format(accuracy_score(y_test, y_pred)))
print('The test error rate is {:.2f}'.format(1 - accuracy_score(y_test, y_pred)))

The accuracy on training data is 1.00
The accuracy on test data is 0.60
The test error rate is 0.40


Training an Ensemble model: train different independent models on slightly different subsets of data
* How to make each model independent with others? 
* Hint: The way the data is fed into the models can be challenging

## Bagging
* Trained Multiple Models on Bootstrap datasets
    + Bootstrap: Resampling the same size of sample with replacement; reduce variance
    + Bagging (or Bootstrap aggregation): agggregate the prediction over a collection of bootstrap samples
    + A Bootstrap sample $\mathbf{Z}^{* b}, b=1,2, \ldots, B$ -> a fitted model $\hat{f}^{* b}(x)$
        $$\hat{f}_{\mathrm{bag}}(x)=\frac{1}{B} \sum_{b=1}^B \hat{f}^{* b}(x)$$

Bagging Inference
* Voting for classification: $\hat{G}_{\text {bag }}(x)=\arg \max _k \hat{f}_{\mathrm{bag}}(x)$
    + "It is tempting to treat the voting proportions pk(x) as estimates of these probabilities. A simple two-class example shows that they fail in this regard." (Hastie, 2008) **How/why fails?**
     <!--  Suppose the true probability of class 1 at x is 0.75, and each of the bagged classifiers accurately predict a 1. Then p1(x) = 1, which is incorrect. -->
    + "An alternative bagging strategy is to average these instead, rather than the vote indicator vectors."
* Averaging for regression 
* Out-of-bag samples: about 1/3 original data is not in the bootstrap dataset which can be used for model evaluation

Goal: reduce the variance of unstable (high variance) learning methods. Assuming that the variables are simply i.d. (identically distributed, but not necessarily independent) with positive pairwise correlation ρ, the variance
of the average is
\begin{equation}
\rho \sigma^2+\frac{1-\rho}{B} \sigma^2
(\#eq:variance)
\end{equation}

<!-- Question 1: Which learning model/method is ideal for bagging?

Question 2: Will it reduce bias?

"since each tree generated in bagging is [identically distributed (i.d.)](https://stats.stackexchange.com/questions/89036/why-the-trees-generated-via-bagging-are-identically-distributed#:~:text=Bagging%20technique%20uses%20bootstraps%20\(random,population%20as%20the%20original%20sample.), the expectation of an average of B such trees is the same as the expectation of any one of them. This means the bias of bagged trees is the same as that of the individual trees, and the only hope of improvement is through variance reduction. This is in contrast to boosting, where the trees are grown in an adaptive way to remove bias, and hence are not i.d." (Hastie, 2008) -->




In [ ]:
# bootstrap sample
def bootstrap():
    bootstrap_sample = []
    bootstrap_label = []
    for j in range(30):
        idx = np.random.randint(0, 30)
        bootstrap_sample.append(X_train[idx])
        bootstrap_label.append(y_train[idx])
    return np.array(bootstrap_sample), np.array(bootstrap_label)

#  generate 200 bootstrap samples
bootstrap_samples = []
bootstrap_labels = []
n_estimators = 200
for i in range(n_estimators):
    bootstrap_sample, bootstrap_label = bootstrap()
    bootstrap_samples.append(bootstrap_sample)
    bootstrap_labels.append(bootstrap_label)

# fit an ensemble of classification trees
ensemble_clf = [DecisionTreeClassifier(random_state=1,).fit(Xtrain, ytrain) for Xtrain, ytrain in zip(bootstrap_samples, bootstrap_labels)]

# predict the test data
y_preds = np.array([clf.predict(X_test) for clf in ensemble_clf])
y_pred = (np.mean(y_preds, axis=0) > 0.5).astype(int)

y_train_pred = y_preds = np.array([clf.predict(X_train) for clf in ensemble_clf])
y_train_pred = (np.mean(y_train_pred, axis=0) > 0.5).astype(int)


<center><img src="pics/bagging.png" width="500"></center>

<center><img src="pics/bagging_result.png" width="500"></center>

* the trees have high variance due to the correlation in the predictors
* Bagging succeeds in smoothing out this variance and hence reducing the test error
    + " averaging reduces variance and leaves bias unchanged" (Hastie, 2008)

## [Random Forest](https://link.springer.com/article/10.1023/A:1010933404324)
* "the size of the correlation of pairs of bagged trees limits the benefits of averaging" according to Formula 1
* "a substantial modification of bagging that builds a large collection of de-correlated trees"
* Iteratively 1) make a bootstrapped dataset; 2) only use a random subset of variables at each splitting (`max_features`)
* can handle large data sets with higher dimensionality (thousands of input variables).
* can identify most significant variables

In [8]:
rf_clf = RandomForestClassifier(n_estimators=n_estimators, criterion="gini", max_features=None) #  max_depth=5,
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(rf_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(rf_clf.score(X_test, y_test)))
print('The test error rate is {:.2f}'.format(1 - rf_clf.score(X_test, y_test)))


# for estimator in rf_clf.estimators_:
#     plt.figure(figsize=(12,12))
#     tree.plot_tree(estimator, filled=True, rounded=True)
#     plt.show()


The accuracy on training data is 1.00
The accuracy on test data is 0.74
The test error rate is 0.26
              precision    recall  f1-score   support

         0.0       0.77      0.85      0.81      1249
         1.0       0.69      0.57      0.63       751

    accuracy                           0.74      2000
   macro avg       0.73      0.71      0.72      2000
weighted avg       0.74      0.74      0.74      2000



[Looking for good `n_estimators`](https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html)

## Boosting
What to boost?

* AdaBoost
    + an fix-sized estimator uses only one feature, i.e., one stump (one root node with two leaf nodes)
    + weak learner
    + build the subsequent stumps using the residuals
    + the amount of say 
        $$\alpha_m=\log \left(\left(1-\operatorname{err}_m\right) / \operatorname{err}_m\right)$$

    + Update the weights
        $$w_i \cdot \exp \left[\alpha_m \cdot I\left(y_i \neq G_m\left(x_i\right)\right)\right], i=1,2, \ldots, N$$
<!-- Emphasize the need to correctly classify the examples with wrong predictions in the previous steps -->

In [130]:
# build an Adaboost classifier with 200 trees
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)
y_pred = ada_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(ada_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(ada_clf.score(X_test, y_test)))
print('The test error rate is {:.2f}'.format(1 - ada_clf.score(X_test, y_test)))
print(classification_report(y_test, y_pred))


The accuracy on training data is 1.00
The accuracy on test data is 0.69
The test error rate is 0.31
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73      1197
         1.0       0.61      0.66      0.63       803

    accuracy                           0.69      2000
   macro avg       0.68      0.69      0.68      2000
weighted avg       0.70      0.69      0.69      2000




* GradientBoost
    + an fixeds-size estimator normally has 8 to 32 leaves
    + iteratively fit residuals by a split
    + use learning rate to avoid high bias
    + [Youtube Course](vhttps://www.youtube.com/watch?v=3CC4N4z3GJc)

In [131]:
# build a Gradient Boosting classifier with 200 trees
gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.5, max_depth=1, random_state=0)
gb_clf.fit(X_train, y_train)
y_pred = gb_clf.predict(X_test)
print('The accuracy on training data is {:.2f}'.format(gb_clf.score(X_train, y_train)))
print('The accuracy on test data is {:.2f}'.format(gb_clf.score(X_test, y_test)))
print('The test error rate is {:.2f}'.format(1 - gb_clf.score(X_test, y_test)))
print(classification_report(y_test, y_pred))



The accuracy on training data is 1.00
The accuracy on test data is 0.69
The test error rate is 0.31
              precision    recall  f1-score   support

         0.0       0.76      0.71      0.73      1197
         1.0       0.60      0.66      0.63       803

    accuracy                           0.69      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.69      0.69      0.69      2000



In [ ]:
# # read titanic data
# titanic_df = pd.read_csv('data/titanic_cleaned_data.csv')
# print(titanic_df.columns)
# # define X and y
# # Port of Embarkation: Q = Queenstown, S = Southampton
# feature_cols = ['Pclass', 'Sex', 'Embarked_Q', 'Embarked_S', 'Age', 'PassengerId']
# X = titanic_df[feature_cols]
# y = titanic_df.Survived
# X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
# ## Finding Important Features
# importances = rf_clf.feature_importances_
# print(importances)
# forest_importances = pd.Series(importances, index=list(X_train.columns))
# std = np.std([tree.feature_importances_ for tree in rf_clf.estimators_], axis=0)
# print(forest_importances.head())

# fig, ax = plt.subplots()
# forest_importances.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

## Pass Activity

In [26]:
# Q1: Load HR-Employee-Attrition.csv dataset and 
# create an ensemble ML model for predicting target variable (Attrition). 
# Report the performance of the model using appropriate metrics.
def load_data():
    df = pd.read_csv("data/HR-Employee-Attrition.csv")
    df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis="columns", inplace=True)
    categorical_col = []
    for column in df.columns:
        if df[column].dtype == object and len(df[column].unique()) <= 50:
            categorical_col.append(column)
    print(f'The {len(categorical_col)} features will be used: {categorical_col}.')
    print('The values for Attrition column are: ')
    print(df.Attrition.value_counts())
    df['Attrition'] = df.Attrition.astype("category").cat.codes
    print('The Attrition column has been encoded. The values are: ')
    print(df.Attrition.value_counts())
    
    return df

df = load_data()
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=5)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
print(classification_report(y_test, pred))
print(classification_report(y_train, rf_clf.predict(X_train)))

# Q2: Have you used any hyperparameter tuning while building the model in Q1? 
# If so then plot your performance metrics for different hyperparmeter values that you have used in Q1.  Hints

# Q3: Reflect on the importance of hyperparameter tuning of ML models based on your ML model development exercise. 

# 4. Create a GradientBoost model for the predicting Attrition using the same dataset that you have used Q1 and report the performance.

# 5. Compare the performance of two models (Q1 and Q3). Explain which model is good and why.

The 8 features will be used: ['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime'].
The values for Attrition column are: 
No     1233
Yes     237
Name: Attrition, dtype: int64
The Attrition column has been encoded. The values are: 
0    1233
1     237
Name: Attrition, dtype: int64
              precision    recall  f1-score   support

           0       0.78      0.92      0.85       157
           1       0.85      0.63      0.73       111

    accuracy                           0.80       268
   macro avg       0.82      0.78      0.79       268
weighted avg       0.81      0.80      0.80       268

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       392
           1       0.92      0.68      0.78       231

    accuracy                           0.86       623
   macro avg       0.88      0.82      0.84       623
weighted avg       0.87      0.86      0.85       623

